In [130]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain, TransformChain
import numpy as np

In [122]:
#creating rett.txt
from PyPDF2 import PdfReader
import fitz
import pandas as pd

#reader = PdfReader('rettsyndrome.pdf')

doc = fitz.open('rettsyndrome.pdf')
count = doc.page_count
#textList = []
with open('rett.txt', 'a') as f:
    for i in range(0, count-1):
        x = doc[i].get_text()
        f.write(x)


In [50]:
#creating rett2.txt
import PyPDF2
with open('rettsyndrome.pdf','rb') as f:
    pdf_reader = PyPDF2.PdfReader(f)
    text = ''
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        text += page.extract_text()

with open('rett2.txt', 'w',encoding='utf-8') as file:
    file.write(text)


In [8]:
from langchain.llms import OpenAI
import os
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
llm = OpenAI(temperature =0)

In [96]:
# chunk rett.txt; run each chunk thru gpt for summary -> df of summary; doc chunk; embedding
# read .txt in chunks; get summary, return tuple, withid; write each tuple to csv
#TODO: would be easier if sentences ended at periods - instead of chunk size, find farthest period within chunk_size
import codecs
chunk_dict = {}
def read_in_chunks(file, chunk_size = 1000,index=0):
    with codecs.open(file,'r',encoding='utf-8',errors='ignore') as f:
        while True:
            f.seek(index)
            data = f.read(chunk_size)
            if not data:
                break
            return data

In [97]:
x = read_in_chunks('rett2.txt', 800,0)
#print(x)
type(x)

str

### could finetune a model to create headers

In [82]:
# function to summarize chunk -
llm = OpenAI(temperature=0)  # model = "text-ada-001",
def summarize_chunk(text):
    prompt= f"Create a 4-word header for the following text: {text}"
    summary = llm(prompt)
    return summary
    


In [83]:
text = 'Rett syndrome(RTT; OMIM identifier  # 312750), first reported by the Austrian physician Andreas Rett, is a devastating neurological disorder that affects brain development and function in females in approximately 1 in 10, 000 live births 1, 2. In 90–95 % of patients diagnosed with classic RTT, the disease is caused by loss - of-function mutations in the X - linked MECP2 gene, which encodes methyl - CpG binding protein 2 (MeCP2).MECP2 gene mutations predominantly arise in germ cells from the paternal side3. Despite the hundreds of RTT - causing MECP2 mutations that have been identified, eight ‘hotspot’ mutations(encoding the amino acid substitutions R106W, R133C, T158M, R168X, R255X, R270X, R294X and R306C) constitute more than 60 % of documented cases4. RTT -causing mutations in MECP2 in female patients lead to developmental regression, including a loss of speech and hand skills, after apparently normal development5. In females, random X chromosome inactivation results in somatic mosaics with normal and mutant'
summarize_chunk(text)

' MECP2 alleles in different cells6.\n\nDevastating Neurological Disorder:'

In [114]:
import re
file = 'rett2.txt'

def iterate_chunks(file, chunk_size):
    list_of_lists = []
    list_of_lists.append(['chunk', 'summary', 'index'])
    index = 0
    for chunk in read_in_chunks(file):
    #chunk is starting at the same index every time
        x = read_in_chunks(file, chunk_size,index=index)
        #x= re.sub(r'[^\x00-\x7f]+','', x)
        summ = 'h'
        zip = [x, summ, index]
        list_of_lists.append(zip)
        index += chunk_size
    return list_of_lists
l = iterate_chunks(file, chunk_size=1000)


In [118]:
import csv
disease_name='rett'
output = f'{disease_name}output.csv'
# Open a CSV file for writing
with open(output, 'w', newline='') as f:
    # Create a CSV writer object
    writer = csv.writer(f)
    # Write the data to the CSV file
    writer.writerows(l)


In [119]:
df= pd.read_csv(output)
df.head()

,chunk,summary,index
0,"Rett syndrome (RTT; OMIM identifier #312750), ...",h,0
1,esults in somatic mosaics with normal and mut...,h,1000
2,"velopmental stages, MeCP2–DNA interactions, th...",h,2000
3,"red verbal skills. In addition, normal hand fu...",h,3000
4,Nikolaos Mellios2 and Mriganka Sur 1*\nAbstra...,h,4000


## Add embeddings

In [120]:
print(len(df))

1000


In [ ]:
def get_embedding(text: str, model: str = EMBEDDING_MODEL) -> list[float]:
    result = openai.Embedding.create(
        model=model,
        input=text
    )
    return result["data"][0]["embedding"]


def compute_doc_embeddings(df: pd.DataFrame) -> dict[tuple[str, str], list[float]]:
    """
    Create an embedding for each row in the dataframe using the OpenAI Embeddings API.
    
    Return a dictionary that maps between each embedding vector and the index of the row that it corresponds to.
    """
    return {
        idx: get_embedding(r.content) for idx, r in df.iterrows()
    }


In [ ]:
def load_embeddings(fname: str) -> dict[tuple[str, str], list[float]]:
    """
    Read the document embeddings and their keys from a CSV.
    
    fname is the path to a CSV with exactly these named columns: 
        "title", "heading", "0", "1", ... up to the length of the embedding vectors.
    """

    df = pd.read_csv(fname, header=0)
    max_dim = max([int(c)
                  for c in df.columns if c != "title" and c != "heading"])
    return {
        (r.title, r.heading): [r[str(i)] for i in range(max_dim + 1)] for _, r in df.iterrows()
    }


In [ ]:
document_embeddings = compute_doc_embeddings(df)

## Similarity search

In [ ]:
def vector_similarity(x: list[float], y: list[float]) -> float:
    """
    Returns the similarity between two vectors.
    
    Because OpenAI Embeddings are normalized to length 1, the cosine similarity is the same as the dot product.
    """
    return np.dot(np.array(x), np.array(y))


def order_document_sections_by_query_similarity(query: str, contexts: dict[(str, str), np.array]) -> list[(float, (str, str))]:
    """
    Find the query embedding for the supplied query, and compare it against all of the pre-calculated document embeddings
    to find the most relevant sections. 
    
    Return the list of document sections, sorted by relevance in descending order.
    """
    query_embedding = get_embedding(query)

    document_similarities = sorted([
        (vector_similarity(query_embedding, doc_embedding), doc_index) for doc_index, doc_embedding in contexts.items()
    ], reverse=True)

    return document_similarities


In [ ]:
order_document_sections_by_query_similarity(
    "Who won the men's high jump?", document_embeddings)[:5]


## Add relevant sections to context in prompt

## Default questions


In [124]:
default_questions = """
'What is the mechanism of action of {disease}?'
'What is the molecular basis for {disease}?'
'What is the pathophysiology of {disease}?'
'What are the clinical manifestations of {disease}?'
'What are current treatments for {disease}?'
'{disease} epidemiology'
'{disease} transcriptomics'

"""

In [ ]:
# ask question, put response in memory

In [126]:
default_template = """ 
You are a top scientist. You want to answer {question} factually and truthfully. In order to answer {question}, you need to search through your knowledge base. If the answer doesn't exist in your knowledge base, you need to look it up.

"""

In [ ]:
# chain should output search query

In [ ]:
# default prompt
default_prompt = PromptTemplate(
    input_variables = ["question"],
    template = default_template,
)
# this question should search embedded db; opentargets -> return a bunch of targets and characteristics stored in entity memory
question1 = "What is the molecular basis for Rett syndrome?"
chain = LLMChain(llm=llm, prompt=default_prompt)
chain.run("question1")

In [200]:
question_gen_template = """ 
{information}
From the above information, formulate a useful hypothesis and phrase it as a question
"""
question_prompt = PromptTemplate(
    input_variables = ["information"],
    template = question_gen_template,
)
question_gen_chain = LLMChain(llm = llm, prompt = question_prompt,output_key = "question")

In [202]:
from abc import ABC, abstractmethod
from typing import Any, Dict, List, Optional, Tuple

from pydantic import BaseModel, Field
from langchain.chains.base import Chain
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter, TextSplitter


In [132]:
# write sample result to entity memory, check contents of entity memory
from langchain.memory import ConversationEntityMemory

In [134]:
memory = ConversationEntityMemory(llm=llm)
_input = {'input':"Deven & Sam are working on a hackathon project"}
memory.load_memory_variables(_input)
memory.save_context(
    _input, 
    {"output": "Great! What kind of project are they working on?"}
)

In [136]:
from langchain.memory.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE
from langchain.chains import ConversationChain
from pydantic import BaseModel
from typing import List, Dict, Any

In [137]:
conversation = ConversationChain(
    llm = llm,
    verbose=True,
    prompt = ENTITY_MEMORY_CONVERSATION_TEMPLATE,
    memory = ConversationEntityMemory(llm=llm)
)

In [149]:
templateprompt = """ 
Define the following term: {term}
"""
pro = PromptTemplate(
    input_variables = ["term"],
    template = templateprompt
)
test = LLMChain(
    llm=llm,
    prompt = pro,
    memory = ConversationEntityMemory(llm=llm)
)

In [151]:
test("Rett syndrome")

{'term': 'Rett syndrome',
 'history': 'Human: term\nAI: \nTerm is a word or phrase used to describe a specific concept or idea. It is often used in the context of a particular field or area of study.',
 'entities': {'Rett Syndrome': ''},
 'text': '\nRett syndrome is a rare genetic neurological disorder that affects the development of the brain in girls. It is characterized by a slowing of development, loss of purposeful use of the hands, problems with walking, seizures, and intellectual disability.'}

In [152]:
test.memory.store

{'Rett Syndrome': 'Rett syndrome is a rare genetic neurological disorder that affects the development of the brain in girls, characterized by a slowing of development, loss of purposeful use of the hands, problems with walking, seizures, and intellectual disability.'}

In [154]:
from pprint import pprint
pprint(test.memory.store)

{'Rett Syndrome': 'Rett syndrome is a rare genetic neurological disorder that '
                  'affects the development of the brain in girls, '
                  'characterized by a slowing of development, loss of '
                  'purposeful use of the hands, problems with walking, '
                  'seizures, and intellectual disability.'}


In [155]:
test("transcriptome")

{'term': 'transcriptome',
 'history': 'Human: term\nAI: \nTerm is a word or phrase used to describe a specific concept or idea. It is often used in the context of a particular field or area of study.\nHuman: Rett syndrome\nAI: \nRett syndrome is a rare genetic neurological disorder that affects the development of the brain in girls. It is characterized by a slowing of development, loss of purposeful use of the hands, problems with walking, seizures, and intellectual disability.',
 'entities': {'Transcriptome': ''},
 'text': '\nThe transcriptome is the set of all RNA molecules, including mRNA, rRNA, tRNA, and other non-coding RNA, that are produced by the transcription of DNA in a cell or organism.'}

In [156]:
test.memory.store

{'Rett Syndrome': 'Rett syndrome is a rare genetic neurological disorder that affects the development of the brain in girls, characterized by a slowing of development, loss of purposeful use of the hands, problems with walking, seizures, and intellectual disability.',
 'Transcriptome': 'The transcriptome is the set of all RNA molecules, including mRNA, rRNA, tRNA, and other non-coding RNA, that are produced by the transcription of DNA in a cell or organism.'}

In [157]:
pprint(test.memory.store)

{'Rett Syndrome': 'Rett syndrome is a rare genetic neurological disorder that '
                  'affects the development of the brain in girls, '
                  'characterized by a slowing of development, loss of '
                  'purposeful use of the hands, problems with walking, '
                  'seizures, and intellectual disability.',
 'Transcriptome': 'The transcriptome is the set of all RNA molecules, '
                  'including mRNA, rRNA, tRNA, and other non-coding RNA, that '
                  'are produced by the transcription of DNA in a cell or '
                  'organism.'}


In [195]:
prompt2 = """
Extract the entity from the following text according to this example: 
Text: MECP2 is a transcriptional regulator that is mutated in Rett syndrome.
Entity: MECP2

Text: {text}
Entity:
"""
prompttemplate = PromptTemplate(
    input_variables = ["text"],
    template = prompt2
)
querychain = LLMChain(
    llm=llm, 
    prompt=prompttemplate,
    memory=ConversationEntityMemory(llm=llm),
    output_key = "entity"
)

In [163]:
querychain.memory.store

{'BDNF': 'BDNF is a transcription factor protein involved in neuronal maturity and synaptic plasticity.'}

In [170]:
question_to_query_chain("What is the molecular basis for Rett syndrome?")

{'question': 'What is the molecular basis for Rett syndrome?',
 'text': '"Molecular basis for Rett syndrome"'}

In [171]:
testquery = """ 
Turn the following query: {query} into a SERPAPI api call.
"""
qprompt = PromptTemplate(
    input_variables = ["query"],
    template = testquery
)
chain2 = LLMChain(llm=llm, prompt=qprompt,output_key = "que")

In [174]:
#chain2("Molecular basis for Rett syndrome")
question = question_to_query_chain("What is the molecular basis for Rett syndrome?").get('text')


In [175]:
from serpapi import GoogleSearch
api_key = os.environ['SERPAPI_API_KEY'] = "40f6fba6120a66c379436fe451a4117f8ffea1edffbb935fef773be799b9bac4"


In [183]:
params = {
    "engine": "google",
    "q": question,
    "api_key": api_key
}

search = GoogleSearch(params)
results = search.get_dict()
rs = results["organic_results"]
print(rs)


[{'position': 1, 'title': 'Structure of the MeCP2–TBLR1 complex reveals a ... - PNAS', 'link': 'https://www.pnas.org/content/114/16/E3243', 'displayed_link': 'https://www.pnas.org › content', 'thumbnail': 'https://serpapi.com/searches/641ba56e9d1586ed4adf1b96/images/0fe908a30a7fe1944260d218b25dae0d886a93f0d0048529cd085feb099d1665.jpeg', 'snippet': 'Structure of the MeCP2–TBLR1 complex reveals a molecular basis for Rett syndrome and related disorders. Valdeko Kruusvee, Matthew J. Lyst, Ceitidh Taylor, ...', 'snippet_highlighted_words': ['molecular basis for Rett syndrome'], 'rich_snippet': {'top': {'extensions': ['by V Kruusvee', '2017', 'Cited by 61 —'], 'detected_extensions': {'unknown': 2017, 'cited_by': 61}}}, 'about_this_result': {'source': {'description': 'Proceedings of the National Academy of Sciences of the United States of America is a peer-reviewed multidisciplinary scientific journal. It is the official journal of the National Academy of Sciences, published since 1915, and p

In [185]:
string_comprehension = """" 
Rett syndrome (RTT) is an X-linked neurological disorder caused by mutations in the methyl-CpG–binding protein 2 (MeCP2) gene. The majority of RTT missense mutations disrupt the interaction of the MeCP2 with DNA or the nuclear receptor corepressor (NCoR)/silencing mediator of retinoic acid and thyroid receptors (SMRT) corepressor complex. Here, we show that the “NCoR/SMRT interaction domain” (NID) of MeCP2 directly contacts transducin beta-like 1 (TBL1) and TBL1 related (TBLR1), two paralogs that are core components of NCoR/SMRT. We determine the cocrystal structure of the MeCP2 NID in complex with the WD40 domain of TBLR1 and confirm by in vitro and ex vivo assays that mutation of interacting residues of TBLR1 and TBL1 disrupts binding to MeCP2. Strikingly, the four MeCP2-NID residues mutated in RTT are those residues that make the most extensive contacts with TBLR1. Moreover, missense mutations in the gene for TBLR1 that are associated with intellectual disability also prevent MeCP2 binding. Our study therefore reveals the molecular basis of an interaction that is crucial for optimal brain function.
"""

In [186]:
comp_template = """ 
{text}
From this text, answer this question: {question}
"""

comp_prompt = PromptTemplate(
    input_variables = ["text", "question"],
    template = comp_template
)

## dumbest end to end version:

In [ ]:
# question -> query -C1
# query -> api -C1 (or could be a script)
# result -> entity memory
# entity memory -> new question {its own script - question formulator}; this could benefit from finetuning later

In [27]:
from langchain.memory import SimpleMemory
question = input("Enter your question:")
simp_mem = SimpleMemory()
simp_mem.memories = {"question": question}

In [28]:
simp_mem.memories

{'question': 'test q'}

In [29]:
def return_memories(memory):
    return memory.memories

In [30]:
return_memories(simp_mem)

{'question': 'test q'}

In [213]:
q2q_prompt_template = """ 
Turn the following question into a search query: {question}

"""
# takes in question, returns search query
q2q_prompt = PromptTemplate(
    input_variables = ["question"],
    template = q2q_prompt_template,
)

question_to_query_chain = LLMChain(llm=llm, prompt=q2q_prompt,output_key="query")

In [217]:
memory = ConversationEntityMemory(llm=llm)
query_prompt_template = """ 
Turn this query: {query} into an API call to SerpAPI
"""

query_prompt = PromptTemplate(
    input_variables=["query"],
    template=query_prompt_template,
)

query_api_chain = LLMChain(
    llm=llm,
    prompt=query_prompt,
    output_key="api_call",
)


In [222]:
# link these two chains + memory
memory = ConversationEntityMemory(llm=llm)
total_chain = SequentialChain(
    chains=[question_to_query_chain, query_api_chain],
    input_variables=["question"],
    output_variables=["query"],
    memory=memory
)


In [278]:
# testing above chain 
question = "What is the molecular basis for Rett syndrome?"
query = total_chain(question)
#type(query)

In [230]:
# turn query into serpapi call
from serpapi import GoogleSearch

search_q = query.get("query")
url = "https://serpapi.com/search.json?q={search_q}"

params = {
    "q": search_q,
    "engine":"google",
    "api_key": api_key
}

search = GoogleSearch(params)
results = search.get_dict()
organic_results = results["organic_results"]


In [236]:
print(results)
results.get('menu_items')
# get titles, descriptions, urls -> df
# embed titles in df, match titles to query || ask chatgpt to pick the academic paper

{'search_metadata': {'id': '641d43de303eb19e3ffacd65', 'status': 'Success', 'json_endpoint': 'https://serpapi.com/searches/ea71e89cb9cf19c1/641d43de303eb19e3ffacd65.json', 'created_at': '2023-03-24 06:31:58 UTC', 'processed_at': '2023-03-24 06:31:58 UTC', 'google_url': 'https://www.google.com/search?q=%22Molecular+basis+for+Rett+syndrome%22&oq=%22Molecular+basis+for+Rett+syndrome%22&sourceid=chrome&ie=UTF-8', 'raw_html_file': 'https://serpapi.com/searches/ea71e89cb9cf19c1/641d43de303eb19e3ffacd65.html', 'total_time_taken': 0.82}, 'search_parameters': {'engine': 'google', 'q': '"Molecular basis for Rett syndrome"', 'google_domain': 'google.com', 'device': 'desktop'}, 'search_information': {'organic_results_state': 'Results for exact spelling', 'query_displayed': '"Molecular basis for Rett syndrome"', 'total_results': 3700, 'time_taken_displayed': 0.47, 'menu_items': [{'position': 1, 'title': 'Images', 'link': 'https://www.google.com/search?q=%22Molecular+basis+for+Rett+syndrome%22&sourc

## tried getting results from answer box, for simplest version use serpapi wrapper

In [270]:
answer_box = results.get('answer_box')
# list comprehension
#list(answer_box.keys())[list(answer_box.values()).index('Rett')]

snippet = results.get('answer_box').get('hover_info')[0].get('info')
#print(snippet)
print(answer_box)

ValueError: 'Rett' is not in list

## tried using titles and links -> df, embedding df -> similarity search -> pull info from most relevant link

In [251]:
#r = results.get('search_information').get('related_questions')[0].get('title') 
#To fix later: this loop doesn't really work, also related questions isn't the best place to look; try snippets first? semantic scholar?
i=0
titles = []
links = []
while True:
    try:
        title = results.get('related_questions')[i].get('title')
        link = results.get('related_questions')[i].get('link')
        titles.append(title)
        links.append(link)
        i += 1
    except IndexError:
        break
link = results.get('related_questions')[0].get('link')
title = results.get('related_questions')[0].get('title')
print(titles)
print(links)

['What causes Rett syndrome? | NICHD', 'MECP2 gene: MedlinePlus Genetics', 'Rett syndrome: a neurological disorder with metabolic components']
['https://www.nichd.nih.gov/health/topics/rett/conditioninfo/causes#:~:text=Most%20cases%20of%20Rett%20syndrome,located%20on%20the%20X%20chromosome.', 'https://medlineplus.gov/genetics/gene/mecp2/#:~:text=Normal%20Function&text=The%20MECP2%20gene%20provides%20instructions,that%20packages%20DNA%20into%20chromosomes.', 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5830535/#:~:text=Rett%20syndrome%20(RTT)%20is%20a,RTT%20symptoms%20is%20largely%20unknown.']


## try using serpapi wrapper from langchain, important part is writing to entity memory + coming up with better questions. can tweak the search part later

In [5]:
from langchain import SerpAPIWrapper
import os
api_key = os.environ['SERPAPI_API_KEY'] = "40f6fba6120a66c379436fe451a4117f8ffea1edffbb935fef773be799b9bac4"
search_question = "What's the molecular basis for Rett syndrome"
params = {
    "q": search_question,
    'engine':'google',
    'gl':'us'
}
serpapi = SerpAPIWrapper(serpapi_api_key=api_key)
#res = serpapi.run(search_q)
serpapi.run("What's the molecular basis for Rett syndrome?")

'Rett syndrome (RTT) is an X-linked neurological disorder caused by mutations in the methyl-CpG–binding protein 2 (MeCP2) gene.'

In [10]:
# result = get relevant portions of api call (for now just result of serpapi wrapper call above)
#TODO: this chain needs to run async bc it needs to take in the original question (from entity memory?)
llm = OpenAI(temperature=0)
memory = ConversationEntityMemory(llm=llm)

parse_result_template = """ 
Parse the following result: {result} to get the answer to the question: {question}
"""
parse_result_prompt = PromptTemplate(
    input_variables=["result", "question"],
    template = parse_result_template
)

# try getting original question from user input = question
parse_result_chain = LLMChain(
    llm = llm,
    prompt = parse_result_prompt,
    output_key = "answer", # this is what i want to store in memory, not the inputs of the prompt
    #memory=memory # don't want to store inputs, want to store answers next to questions
    #TODO: could also give questions ID's and give answers ID's, then merge memory
)

NameError: name 'PromptTemplate' is not defined

In [319]:
answer = parse_result_chain({"result": res, "question": question})
#issue running multi-input chain -> bc of memory 

In [9]:
# write result to entity memory - separated from chain
from langchain.memory import ConversationEntityMemory
llm = OpenAI(temperature=0)
memory = ConversationEntityMemory(llm=llm)
input = {"input": answer.get('answer')}
memory.load_memory_variables(input)


NameError: name 'answer' is not defined

In [18]:
# write result of ^ to entity memory
memory = ConversationEntityMemory(llm=llm)
answer_prompt_template = """ 
Write the following to entity memory: {answer}"""
answer_prompt = PromptTemplate(
    input_variables=["answer"],
    template=answer_prompt_template,
)
conversation_chain = LLMChain(
    llm=llm, 
    prompt=answer_prompt,
    output_key="ans-to-mem",
    memory=memory
)

In [20]:
ans = "Rett syndrome is caused by mutations in the gene MECP2, which encodes methyl CpG binding protein 2 (MeCP2)."
conversation_chain({"answer": ans})

{'answer': 'Rett syndrome is caused by mutations in the gene MECP2, which encodes methyl CpG binding protein 2 (MeCP2).',
 'history': '',
 'entities': {'MECP2': '', 'MeCP2': ''},
 'ans-to-mem': '\n\nEntity Memory:\nRett Syndrome is a neurological disorder caused by mutations in the gene MECP2, which encodes methyl CpG binding protein 2 (MeCP2).'}

In [23]:
from pprint import pprint
pprint(conversation_chain.memory.store)

{'MECP2': 'MECP2 is a gene that encodes methyl CpG binding protein 2 and is '
          'associated with mutations that cause Rett Syndrome.',
 'MeCP2': 'MeCP2 is a gene that encodes methyl CpG binding protein 2 and is '
          'associated with Rett syndrome.'}


In [24]:
# question generator
#1. what do you know about rett synrome?
memory = conversation_chain.memory.store
entity = "Rett syndrome"
check_memory_template = """ 
Given what you know about {entity}, generate 5 new questions to learn more about it. Use your {memory} stores to help you

"""
check_memory_prompt = PromptTemplate(
    input_variables=["entity","memory"],
    template=check_memory_template,
)
check_memory_chain = LLMChain(
    llm=llm,
    prompt=check_memory_prompt,
    output_key="questions",
)
#2. generate new hypotheses

In [25]:
memory = conversation_chain.memory.store
ans_mem_q_template = """ 
Memory: {memory}
Using your memory, answer this question: {question}
"""
ans_mem_q_prompt = PromptTemplate(
    input_variables = ["question", "memory"],
    template = ans_mem_q_template,
)

ans_mem_q_chain = LLMChain(
    llm= llm, 
    prompt=ans_mem_q_prompt,
    output_key="answer",
)

In [26]:
check_memory_chain({"entity": entity, "memory": memory})

{'entity': 'Rett syndrome', 'memory': {'MECP2': 'MECP2 is a gene that encodes methyl CpG binding protein 2 and is associated with mutations that cause Rett Syndrome.', 'MeCP2': 'MeCP2 is a gene that encodes methyl CpG binding protein 2 and is associated with Rett syndrome.'}, 'questions': '1. What are the most common mutations of the MECP2 gene associated with Rett Syndrome?\n2. How does the MeCP2 gene affect the development of Rett Syndrome?\n3. What are the long-term effects of Rett Syndrome on individuals?\n4. What treatments are available to help manage the symptoms of Rett Syndrome?\n5. Are there any lifestyle changes that can help reduce the severity of Rett Syndrome?'}

In [339]:
# parse questions into list
output = {'entity': 'Rett syndrome',
          'memory': {'MECP2': 'MECP2: A gene encoding methyl-CpG-binding protein 2, which when mutated can cause Rett Syndrome.',
                     'Rett Syndrome': 'Rett Syndrome: Molecular Basis: Mutations in gene encoding methyl-CpG-binding protein 2 (MeCP2).',
                     'MeCP2': 'MeCP2: Gene encoding methyl-CpG-binding protein 2, which is the molecular basis for Rett Syndrome.'},
          'questions': '\n1. What other genes are associated with Rett Syndrome?\n2. How does the mutation of MECP2 affect the function of the protein?\n3. What are the long-term effects of Rett Syndrome?\n4. What are the most common symptoms of Rett Syndrome?\n5. What treatments are available for Rett Syndrome?'}
q_list = []
questions = output.get('questions')
for q in questions.split('\n'):
    q_list.append(q)
q_list.remove('')

In [342]:
print(q_list[0])

1. What other genes are associated with Rett Syndrome?


In [355]:
#first, run the first new question in total_chain
question = q_list[0]
total_chain({"question":question})

{'question': '1. What other genes are associated with Rett Syndrome?',
 'history': 'Human: What is the molecular basis for Rett syndrome?\nAI: "Molecular basis for Rett syndrome"\nHuman: What is the molecular basis for Rett syndrome?\nAI: "Molecular basis for Rett syndrome"\nHuman: What is the molecular basis for Rett syndrome?\nAI: "Molecular basis for Rett syndrome"',
 'entities': {'Rett Syndrome': 'Rett Syndrome is caused by mutations in the MECP2 gene, which affects the regulation of other genes and is associated with neurological impairments.'},
 'query': '"Rett Syndrome associated genes"'}

In [ ]:
# question -> opentargets query -> answer -> entity memory
q_to_opentargets_prompt = """ 
Given the question: {question} and strategy {strategy}, return a query that follows the strategy to parse a graph database of drugs.
"""

q_to_opentargets_prompt = PromptTemplate(
    input_variables=["question"],
    template=q_to_opentargets_prompt
)

q_to_opentargets_chain = LLMChain(
    llm = llm,
    prompt = q_to_opentargets_prompt,
    output_key = "query"
)

In [ ]:
# first test opentarget queries + strategies below: try two hop query for rett syndrome gene  

In [36]:
# also try to return gene of rett syndrome from entity memory

# testing memory store if it's still in github
from langchain.memory import ConversationEntityMemory
memory = ConversationEntityMemory(llm=llm)
input = {"input":"Rett syndrome is caused by mutations in the gene MECP2, which encodes methyl CpG binding protein 2 (MeCP2)."}
memory.store
#TODO: don't want entities and history, want full memory store!

{}